In [33]:
# prompt: Use random forest model to classify email using pipe

#!pip install datasets

from transformers import pipeline
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np


ds = load_dataset("elenigkove/Email_Intent_Classification")

ds2 = load_dataset("aadilsayad/email-intent-classification")

df = pd.read_csv("TrainingData.txt", header=0)

df2 = df.Email.to_frame()

ds_train = Dataset.from_pandas(df2)  # Creating training data

df3 = df.PrimaryAsk.to_frame()

ds_primask_test = Dataset.from_pandas(df3)  # Creating primary ask test data

df4 = df.SecondaryAsk.to_frame()

ds_secask_test = Dataset.from_pandas(df4)  # Creating secondary ask test data

# Use a pipeline as a high-level helper
pipe = pipeline("feature-extraction", model="nicoladecao/msmarco-word2vec256000-distilbert-base-uncased")

#print(ds)

# Example email data and labels (replace with your actual data)
#emails = [
#    "This is a promotional email about a new product.",
#    "Please confirm your appointment for tomorrow.",
#    "We have detected suspicious activity on your account.",
#    "Your order has been shipped.",
#    "Reminder: Meeting at 2 PM today."
#]

# Access the 'train' split of the dataset
train_ds = ds['train']
train_ds2 = ds2['train']

#train_ds3 = ds3.feature

# Extract data from the 'E' (email) column
emails = train_ds['Email']
emails2 = train_ds2['prompt']
emails3 = ds_train

# Extract data from the 'L' (label) column
labels = train_ds['Intent']
labels2 = train_ds2['label']
labels3 = ds_primask_test
labels4 = ds_secask_test

# Now you have the emails and labels as separate lists/arrays
# You can iterate through them or use them for your machine learning tasks

# Example: Print the first 5 emails and their labels
for i in range(len(emails3)):
    print(f"Email: {emails3[i]}, Label: {labels3[i]}, Label2: {labels4[i]}")

#for j in range(len(ds2['train'])):
#    print(f"Email: {emails2[i]}, Label: {labels2[i]}")

#labels = [0, 1, 2, 1, 1]  # Example labels (0: promotional, 1: personal, 2: security)


# Extract features for each email
#features = [np.mean(pipe(email), axis=0) for email in emails]
features = [pipe(email)[0][0] for email in emails + emails2 + emails3['Email']]
features2 = [pipe(email)[0][0] for email in emails3['Email']]


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels + labels2 + labels3['PrimaryAsk'], test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2, labels4['SecondaryAsk'], test_size=0.2, random_state=42)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

rf_classifier2 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier2.fit(X_train2, y_train2)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

y_pred2 = rf_classifier2.predict(X_test2)

# Evaluate the model (e.g., using accuracy, precision, recall, F1-score)
# ... (add your evaluation metrics here)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

# Evaluate the model for sub request type
accuracy2 = accuracy_score(y_test2, y_pred2)
print("Accuracy for sub request type:", accuracy2)
print(classification_report(y_test2, y_pred2))


# Example: Predict the class of a new email
new_email = "I want to unsubscribe from your newsletter."
#new_features = np.mean(pipe(new_email), axis=0)
new_features = pipe(new_email)[0][0]
print(new_features)
predicted_class = rf_classifier.predict([new_features])[0]
predicted_class2 = rf_classifier2.predict([new_features])[0]
print(f"Predicted class for new email: {predicted_class}")
print(f"Predicted secondary class for new email: {predicted_class2}")


Device set to use cpu


Email: {'Email': 'Hello Everyone, Please be informed that Mr X is visiting our office campus today. Individual presence is mandatory in office.'}, Label: {'PrimaryAsk': ' Instruction'}, Label2: {'SecondaryAsk': ' Official Communication'}
Email: {'Email': 'Dear Sir, I would like to foreclose my home loan. Please assist me with the necessary paperworks. Thank you in advance!'}, Label: {'PrimaryAsk': ' Account closure'}, Label2: {'SecondaryAsk': ' Home Loan'}
Email: {'Email': '“Please reset my password.”'}, Label: {'PrimaryAsk': 'password_reset'}, Label2: {'SecondaryAsk': 'account_unlock'}
Email: {'Email': ' "I need help with my order."'}, Label: {'PrimaryAsk': 'order_support'}, Label2: {'SecondaryAsk': 'billing_related'}
Email: {'Email': 'Can you provide me with a refund?'}, Label: {'PrimaryAsk': 'refund'}, Label2: {'SecondaryAsk': 'replacement'}
Email: {'Email': 'My account is locked.'}, Label: {'PrimaryAsk': 'account_locked'}, Label2: {'SecondaryAsk': 'password_reset'}
Email: {'Email':

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m